In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

In [42]:
import pandas as pd
import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

def preprocessing(text):
    extract_except = '[^A-Za-z ]+'
    text = re.sub(extract_except," ",text)
    text = (text.lower()).replace('\t\n'," ") 
    # tokenizing
    token_text = [lemmatizer.lemmatize(word, pos = 'n') for word in word_tokenize(text) if not word in stopwords.words()]

    return token_text

feedback_data = pd.read_csv('Feedback_sheet.csv', encoding = 'ISO-8859-1')
id_feedback = feedback_data[['ID', 'Feedback']]
id_touchpoint = feedback_data[['ID', 'Touchpoint']]




In [3]:
lemmatized_sentences = []
for sentence in feedback_data['Feedback']:
    lemmatized_sentence = preprocessing(sentence)
    lemmatized_sentences.append(" ".join(lemmatized_sentence))
    

In [4]:
feedback_words = pd.DataFrame(lemmatized_sentences, columns = ["Feedback Words"])
feedback_words


,Feedback Words
0,careline line difficult answer fixed
1,atm rhb jasin melaka
2,grade apps
3,maintaining service like gift something good u...
4,hope improve mobile app faction link branch ac...
...,...
3449,branch staff responsive phone call email
3450,customer service rhbgroup customer advocacy rh...
3451,higher interest rate time people need
3452,card support month installment zero interest p...


In [5]:
id_touchpoint
# Cross check different fields
# Google Translate API in Google Sheets
# Looks back for the survey

,ID,Touchpoint
0,bedba74e-1c23-4772-9ca9-6bf81bd1b27f,&lt;item&gt;Branch&lt;/item&gt;
1,e170ee8d-1287-4f22-b597-c1f2d2b3e2cc,"&lt;item&gt;ATM, Cash Deposit Machines&lt;/ite..."
2,feb9fc8d-15c3-407b-961a-e1f09677f3d0,&lt;item&gt;RHB Mobile Banking Application&lt;...
3,22990519-8eb0-4a16-9e07-c9050c50e520,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;ATM...
4,d2122a89-2eef-43e0-b8f8-44d399c04498,&lt;item&gt;RHB Internet Banking&lt;/item&gt;
...,...,...
3449,1c07bad0-288d-456d-a307-09e90d608845,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;Cal...
3450,fdacc075-cd02-40e5-a8a7-c443192562f6,&lt;item&gt;RHB Internet Banking&lt;/item&gt;
3451,c4f22c48-9d7b-4b15-940e-bfcb55497b4a,&lt;item&gt;RHB Internet Banking&lt;/item&gt;
3452,4e86c213-2ed5-40fd-9edf-c9b2a8f229f0,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;Cal...


In [6]:
all_customer_touchpoints = []
for touchpoint in id_touchpoint['Touchpoint']:
    split_touchpoint = touchpoint.split(';')[2:]
    resulting_touchpoint = []
    for item in split_touchpoint:
        if item not in ['item&gt','/item&gt', '&lt', '', '']:
            item = item.replace('&lt','')
            resulting_touchpoint.append(item)
    all_customer_touchpoints.append(resulting_touchpoint)
        


In [7]:
def classify(all_customer_touchpoints):
    all_touchpoint_binary_class = []
    for customer_touchpoint in all_customer_touchpoints:
        touchpoint_binary_class = ["No", "No", "No", "No", "No", "No", "No"]
        for touchpoint in customer_touchpoint:

            if touchpoint == 'Social media':
                touchpoint_binary_class[0] = "Yes"
                continue
            elif touchpoint == 'Call centre':
                touchpoint_binary_class[1] = "Yes"
                continue
            elif touchpoint == 'Branch':
                touchpoint_binary_class[2] = "Yes"
                continue 
            elif touchpoint == 'RHB Internet Banking':
                touchpoint_binary_class[3] = "Yes"
                continue    
            elif touchpoint == 'RHB Mobile Banking Application':
                touchpoint_binary_class[4] = "Yes"
                continue
            elif touchpoint == 'Relationship managers / Personal Bankers':
                touchpoint_binary_class[5] = "Yes"
                continue
            elif touchpoint == 'ATM, Cash Deposit Machines':
                touchpoint_binary_class[6] = "Yes"
                continue

        all_touchpoint_binary_class.append(touchpoint_binary_class)

    return all_touchpoint_binary_class

pd.DataFrame(all_customer_touchpoints)
touchpoint_list = ["Social media", "Call centre", "Branch", "RHB Internet Banking", "RHB Mobile Banking Application", "Relationship managers / Personal Bankers", 'ATM, Cash Deposit Machines']
touchpoint_df = pd.DataFrame(columns = touchpoint_list)
touchpoint_df


all_touchpoint_binary_class = classify(all_customer_touchpoints)
Main_df = pd.DataFrame(all_touchpoint_binary_class, columns =touchpoint_list, dtype = str)
Main_df["Feedback"] = feedback_words
Main_df["Rating"] = feedback_data['Rating']
Main_df

    
            
    
                

            
            
    
    



,Social media,Call centre,Branch,RHB Internet Banking,RHB Mobile Banking Application,Relationship managers / Personal Bankers,"ATM, Cash Deposit Machines",Feedback,Rating
0,No,No,Yes,No,No,No,No,careline line difficult answer fixed,7
1,No,No,No,No,No,No,Yes,atm rhb jasin melaka,6
2,No,No,No,No,Yes,No,Yes,grade apps,6
3,No,No,Yes,No,No,No,Yes,maintaining service like gift something good u...,10
4,No,No,No,Yes,No,No,No,hope improve mobile app faction link branch ac...,5
...,...,...,...,...,...,...,...,...,...
3449,No,Yes,Yes,Yes,No,No,No,branch staff responsive phone call email,6
3450,No,No,No,Yes,No,No,No,customer service rhbgroup customer advocacy rh...,0
3451,No,No,No,Yes,No,No,No,higher interest rate time people need,7
3452,No,Yes,Yes,No,No,No,No,card support month installment zero interest p...,7


In [8]:
Main_df.to_csv("cleaned_dataset.csv")

In [9]:
touchpoint_list

['Social media',
 'Call centre',
 'Branch',
 'RHB Internet Banking',
 'RHB Mobile Banking Application',
 'Relationship managers / Personal Bankers',
 'ATM, Cash Deposit Machines']

TF-IDF Score

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from numpy import dot
from numpy.linalg import norm
import math

In [11]:
feedback_data

,Created,Date,Touchpoint,Rating,Verbatim,Preferred Bank,Product,Feedback,ID
0,8/27/2020 4:42:25 PM,8/27/2020 16:42:24,&lt;item&gt;Branch&lt;/item&gt;,7,"Talian careline susah dijawab, harus diperbaiki",RHB,CASA,"Careline lines are difficult to answer, should...",bedba74e-1c23-4772-9ca9-6bf81bd1b27f
1,8/27/2020 5:22:49 PM,8/27/2020 17:22:33,"&lt;item&gt;ATM, Cash Deposit Machines&lt;/ite...",6,Tiada Atm atau RHB di Jasin Melaka,Maybank,CASA,No ATM or RHB in Jasin Melaka,e170ee8d-1287-4f22-b597-c1f2d2b3e2cc
2,8/27/2020 5:50:18 PM,8/27/2020 17:50:17,&lt;item&gt;RHB Mobile Banking Application&lt;...,6,Up grade your APPS,Affin Bank CIMB,CASA,Up Grade Your Apps,feb9fc8d-15c3-407b-961a-e1f09677f3d0
3,8/27/2020 8:12:22 PM,8/27/2020 20:12:19,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;ATM...,10,Mengekalkan perkhidmatan seperti sekarang dan ...,Rhb bank di petaling jaya ss2.,CASA,Maintaining Services Like Now and Gift Somethi...,22990519-8eb0-4a16-9e07-c9050c50e520
4,8/27/2020 8:14:38 PM,8/27/2020 20:14:36,&lt;item&gt;RHB Internet Banking&lt;/item&gt;,5,hope can improve MOBILE app factions or link b...,ocbc pub,CASA,HOPE CAN IMPROVE MOBILE APP FACTIONS OR LINK B...,d2122a89-2eef-43e0-b8f8-44d399c04498
...,...,...,...,...,...,...,...,...,...
3449,2/9/21 12:53,2/9/21 12:53,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;Cal...,6,branch staff have to be responsive on phone ca...,Maybank,CASA,BRANCH STAFF HAVE TO BE RESPONSIVE ON PHONE CA...,1c07bad0-288d-456d-a307-09e90d608845
3450,2/9/21 0:39,2/9/21 0:39,&lt;item&gt;RHB Internet Banking&lt;/item&gt;,0,customer.service@rhbgroup.com and customer.adv...,Definitely not RHB,Mortgage,Customer.Service@rhbgroup.com and customer.adv...,fdacc075-cd02-40e5-a8a7-c443192562f6
3451,2/9/21 16:46,2/9/21 16:46,&lt;item&gt;RHB Internet Banking&lt;/item&gt;,7,Higher interest rate at times when rakyat need...,Public Bank,FD,Higher Interest Rate at Times WHEN PEOPLE NEED...,c4f22c48-9d7b-4b15-940e-bfcb55497b4a
3452,3/9/21 19:49,3/9/21 19:49,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;Cal...,7,Card does not support 36 months instalment zer...,Maybank and public bank for local bank HSBC fo...,Credit Card,Card Does Not Support 36 Months Installment Ze...,4e86c213-2ed5-40fd-9edf-c9b2a8f229f0


In [64]:
list_of_banks = {"uob": 0,
                 "affin": 0, 
                 "cimb": 0,
                 "hsbc": 0,
                 "bnm": 0,
                 "bsn": 0,
                 "ocbc": 0,
                 "maybank": 0,
                 "public": 0,
                 "ambank": 0,
                 "rakyat": 0,
                 "alliance": 0,
                 "citibank": 0,
                 "chartered": 0,
                 "islam": 0,
                 "rhb": 0,
                 "leong": 0
                }

preffered_bank = pd.DataFrame() 

preffered_bank["Preferred Bank"] = feedback_data['Preferred Bank']
tokenized_preffered_bank = []

for bank in feedback_data['Preferred Bank']:
    tokenized_bank = preprocessing(str(bank))
    
    for bank in tokenized_bank:
        if bank in list_of_banks:
            list_of_banks[bank]+=1
            
list_of_banks


    
    
    
    
    
    

{'uob': 45,
 'affin': 29,
 'cimb': 407,
 'hsbc': 98,
 'bnm': 2,
 'bsn': 24,
 'ocbc': 51,
 'maybank': 1086,
 'public': 319,
 'ambank': 73,
 'rakyat': 72,
 'alliance': 21,
 'citibank': 57,
 'chartered': 28,
 'islam': 118,
 'rhb': 1670,
 'leong': 139}

In [65]:
Bank_hashmap = pd.DataFrame(list_of_banks.items(), columns = ["Bank", "Count"])
Bank_hashmap.to_csv("preffered_bank_count.csv")